In [1]:
import datetime
import numpy

from scipy.optimize import minimize

from ion_populations import single_ion_population_mass
from coordinates_system_transformation import spherical_to_cartesian, cartesian_to_spherical
from magnetic_field_functions import read_b_model
from longitude_functions import objective_function_rho_0_from_delta_longitude, objective_function_zlimN_from_longitude, delta_longitude_calculation, longitude_MAW_TEB_calculation
from density import rho_z
from alfven_velocity import v_alfven_calculation
from Current_Sheet_values_functions import density_and_scale_height_of_CS_calculation




# First method:
(which allows to reproduce the results shown in Table 2 of Rabia et al. (2025) for $H=0.75$, $1.00$, $1.50$, $2.00$, and $3.00$ $R_{J}$​)

1° Reading of $x_i$, $y_i$, $z_i$, $|B_i|$ values along the magnetic field line instantaneously connected to the moon (based on JRM33+KK2005). With i the index of each point along this field line. Transformed the coordinate in the proper referentiel, so that the CS is properly aligned with z = 0. It can be choosen if the current layer is aligned with the centrifugal, jovicentric or magnetic equator.

2° Determining of $ρ_i$ along the field line, using the equation $ρ_i =ρ_0 * exp^{ - \sqrt{\frac{(r_i - r_0)^2 + z_i^2}{H} }}$ for a given $H$ and $ρ_0$. 

3° Determining of $v_{i}^{Alfvén}$ from these values of $ρ_i$ and $B_i$

4° Determining of travel time $t_{TEB}$ and $t_{MAW}$ --> determination of $\lambda_{TEB}$ and $\lambda_{MAW}$ --> determination of $\Delta \lambda$

5° Repeating steps 2° to 4° with different values of $\rho_0$ to minimise $|\Delta \lambda - \Delta \lambda_{observed}|$.

6° Repeating Step 5° for different values of $H$ to minimise $|\lambda_{TEB} - \lambda_\mathrm{TEB observed}|$ and $|\lambda_{MAW} - \lambda_\mathrm{MAW observed}|$.


In [ ]:
R_J = 71492e3 # m

file_mag_model_north = "Bmodel/Callisto_B_line_N_v2.txt"
file_mag_model_south = "Bmodel/Callisto_B_line_S_v2.txt"

(r_N, theta_N, phi_N, B_total_N) = read_b_model(file_mag_model_north)
(r_S, theta_S, phi_S, B_total_S) = read_b_model(file_mag_model_south)

r_N = r_N*R_J
r_S = r_S*R_J

#B is in nT --> T
B_total_N = B_total_N*1e-9 # T: kg/(A.s^2)
B_total_S = B_total_S*1e-9 # T: kg/(A.s^2)

rotation_rate_callisto =  0.0098 # °/s

lambda_observed_MAW = 297.7
lambda_observed_TEB = 303.05


m_e = 9.109e-31 # kg
keV = 1.602e-16 # J == kg.m^2/s^2

rho_0_volumetric_first_guess = 8000/1e-6*m_e #kg/m^3
r_0 = 26.33*R_J

no_CS_calculation = False

delta_L = 1.14
#delta_L = 1


E_array = numpy.array([10])*keV
z_lim_N_array = numpy.array([0.74, 0.96, 1.03, 1.63, 2.22, 2.8, 3.36])*R_J # correspond to specific H values [0.75, 0.94, 1, 1.5, 2., 2.5, 3.0] for CS aligned with centrifugal equator
#z_lim_N_array = numpy.array(z_lim_N in [0.15,0.46, 0.74, 1.115, 1.77, 2.4, 3.01]) # correspond to specific H values [0.75, 1, 1.21, 1.5, 2., 2.5, 3.0] for CS aligned with Mag equator

(H_array, # Height Scale in Rp
            rho_0_array, #total mass density in the center of the PS in kg.cm^-3
            n_0_ions_array, # ions density in the PS cm^-3
            n_0_electrons, # electrons density in the PS cm^-3
            longitude_TEB, # TEB longitude
            longitude_MAW, # MAW longitude
            rho_at_Moon_array, # total mass density at the moon
            n_at_Moon_ions, # ions density at the moon orbit cm^-3
            n_at_Moon_electrons, # electrons density at the moon orbit cm^-3 
            units
            ) =  density_and_scale_height_of_CS_calculation(r_N, theta_N, phi_N, B_total_N,
                                   r_S, theta_S, phi_S, B_total_S,
                                   E_array,
                                   z_lim_N_array, r_0,
                                   lambda_observed_MAW,
                                   lambda_observed_TEB,
                                   rho_0_volumetric_first_guess = rho_0_volumetric_first_guess,
                                   rotation_rate = rotation_rate_callisto,
                                   jovicentric_equator = False,
                                   centrifugal_equator = True,
                                   magnetic_equator = False,
                                   verbose = True,
                                   disk = False,
                                   torus = True,
                                   delta_L = delta_L)

                                   
    


# Second method:
(yields the best results highlighted in red in Table 2 of Rabia et al. (2025) for $H=0.94 	ext{R}_	ext{J}$):
1° Reading of $x_i$, $y_i$, $z_i$, $|B_i|$ values along the magnetic field line instantaneously connected to the moon (based on JRM33+KK2005). With i the index of each point along this field line. Transformed the coordinate in the proper referentiel, so that the CS is properly aligned with z = 0. It can be choosen if the current layer is aligned with the centrifugal, jovicentric or magnetic equator.

2° Determining of $ρ_i$ along the field line, using the equation $ρ_i =ρ_0 * exp^{ - \sqrt{\frac{(r_i - r_0)^2 + z_i^2}{H} }}$ for a given $H$ and $ρ_0$. 

3° Determining of $v_{i}^{Alfvén}$ from these values of $ρ_i$ and $B_i$

4° Determining of travel time $t_{TEB}$ and $t_{MAW}$ --> determination of $\lambda_{TEB}$ and $\lambda_{MAW}$ --> determination of $\Delta \lambda$

5° Repeating steps 2° to 4° with different values of $\rho_0$ and $H$ to minimise both $|\Delta \lambda - \Delta \lambda_{observed}|$ and $|\lambda_{TEB} - \lambda_\mathrm{TEB observed}|$ or $|\lambda_{MAW} - \lambda_\mathrm{MAW observed}|$.




In [ ]:
R_J = 71492e3 # m

file_mag_model_north = "Bmodel/Callisto_B_line_N_v2.txt"
file_mag_model_south = "Bmodel/Callisto_B_line_S_v2.txt"

(r_N, theta_N, phi_N, B_total_N) = read_b_model(file_mag_model_north)
(r_S, theta_S, phi_S, B_total_S) = read_b_model(file_mag_model_south)

r_N = r_N*R_J
r_S = r_S*R_J

#B is in nT --> T
B_total_N = B_total_N*1e-9 # T: kg/(A.s^2)
B_total_S = B_total_S*1e-9 # T: kg/(A.s^2)


rotation_rate_callisto =  0.0098 # °/s
c=3e8 # m/s

longitude_observed_MAW = 297.7
longitude_observed_TEB = 303.05


m_e = 9.109e-31 # kg
keV = 1.602e-16 # J == kg.m^2/s^2

rho_0_volumetric_first_guess = 4000/1e-6*m_e #kg/m^3
z_lim_N_first_guess = 1.0*R_J
r_0 = 26.33*R_J

no_CS_calculation = False

E_array = numpy.array([10])*keV

m_e = 9.109e-31 # kg
E = E_array[0]
delta_longitude_observed = numpy.abs(longitude_observed_MAW-longitude_observed_TEB)
v = numpy.sqrt(2*E / m_e)
centrifugal_equator = True
jovicentric_equator = False
magnetic_equator = False
MAW = True
TEB = False
disk = False
torus = True
#z_lim_N = z_lim_N_first_guess
B_N = B_total_N
B_S = B_total_S

# Outer minimization function to find the optimal z_lim_N
result = minimize(
    objective_function_zlimN_from_longitude,
    z_lim_N_first_guess,
    args=(r_0, rho_0_volumetric_first_guess,
          r_N, theta_N, phi_N, B_N,
          r_S, theta_S, phi_S, B_S,
          E, rotation_rate_callisto,
          360-phi_N[0], longitude_observed_MAW, longitude_observed_TEB, 
          10*71492e3, # equatorial_radius_lim
          71492e3, # Rp
          MAW, TEB, # MAW=True, TEB=False
          disk, torus,  # disk=False, torus=True
          jovicentric_equator, centrifugal_equator, magnetic_equator),  # jovicentric_equator=False, centrifugal_equator=True, magnetic_equator=False
    method='Nelder-Mead',
    tol=1e-6
)

z_lim_N_optimized = result.x


(H_array, # Height Scale in Rp
            rho_0_array, #total mass density in the center of the PS in kg.cm^-3
            n_0_ions_array, # ions density in the PS cm^-3
            n_0_electrons, # electrons density in the PS cm^-3
            longitude_TEB, # TEB longitude
            longitude_MAW, # MAW longitude
            rho_at_Moon_array, # total mass density at the moon
            n_at_Moon_ions, # ions density at the moon orbit cm^-3
            n_at_Moon_electrons, # electrons density at the moon orbit cm^-3 
            units
            ) = density_and_scale_height_of_CS_calculation(r_N, theta_N, phi_N, B_total_N,
                                   r_S, theta_S, phi_S, B_total_S,
                                   E_array,
                                   z_lim_N_optimized, r_0,
                                   longitude_observed_MAW,
                                   longitude_observed_TEB,
                                   rho_0_volumetric_first_guess = rho_0_volumetric_first_guess,
                                   rotation_rate = rotation_rate_callisto,
                                   jovicentric_equator = False,
                                   centrifugal_equator = True,
                                   magnetic_equator = False,
                                   verbose = True,
                                   disk = False,
                                   torus = True)


**Adding a $\delta L = 14$% for the length of the magnetic field line in the CS (which corresponds to $L_{new} = L/cos(\theta) = L*1.14$, with $\theta = arctan(M_A)$ and $M_A = 0.55$)**

In [ ]:
R_J = 71492e3 # m

file_mag_model_north = "Bmodel/Callisto_B_line_N_v2.txt"
file_mag_model_south = "Bmodel/Callisto_B_line_S_v2.txt"

(r_N, theta_N, phi_N, B_total_N) = read_b_model(file_mag_model_north)
(r_S, theta_S, phi_S, B_total_S) = read_b_model(file_mag_model_south)

r_N = r_N*R_J
r_S = r_S*R_J

#B is in nT --> T
B_total_N = B_total_N*1e-9 # T: kg/(A.s^2)
B_total_S = B_total_S*1e-9 # T: kg/(A.s^2)


rotation_rate_callisto =  0.0098 # °/s
c=3e8 # m/s

longitude_observed_MAW = 297.7
longitude_observed_TEB = 303.05


m_e = 9.109e-31 # kg
keV = 1.602e-16 # J == kg.m^2/s^2

rho_0_volumetric_first_guess = 4000/1e-6*m_e #kg/m^3
z_lim_N_first_guess = 1.0*R_J
r_0 = 26.33*R_J

delta_L = 1.14
no_CS_calculation = False

E_array = numpy.array([10])*keV

m_e = 9.109e-31 # kg
E = E_array[0]
delta_longitude_observed = numpy.abs(longitude_observed_MAW-longitude_observed_TEB)
v = numpy.sqrt(2*E / m_e)
centrifugal_equator = True
jovicentric_equator = False
magnetic_equator = False
MAW = True
TEB = False
disk = False
torus = True
#z_lim_N = z_lim_N_first_guess
B_N = B_total_N
B_S = B_total_S

# Outer minimization function to find the optimal z_lim_N
result = minimize(
    objective_function_zlimN_from_longitude,
    z_lim_N_first_guess,
    args=(r_0, rho_0_volumetric_first_guess,
          r_N, theta_N, phi_N, B_N,
          r_S, theta_S, phi_S, B_S,
          E, rotation_rate_callisto,
          360-phi_N[0], longitude_observed_MAW, longitude_observed_TEB, 
          10*71492e3, # equatorial_radius_lim
          71492e3, # Rp
          MAW, TEB, # MAW=True, TEB=False
          disk, torus,  # disk=False, torus=True
          jovicentric_equator, centrifugal_equator, magnetic_equator,
          delta_L),  # jovicentric_equator=False, centrifugal_equator=True, magnetic_equator=False
    method='Nelder-Mead',
    tol=1e-6
)

z_lim_N_optimized = result.x


(H_array, # Height Scale in Rp
            rho_0_array, #total mass density in the center of the PS in kg.cm^-3
            n_0_ions_array, # ions density in the PS cm^-3
            n_0_electrons, # electrons density in the PS cm^-3
            longitude_TEB, # TEB longitude
            longitude_MAW, # MAW longitude
            rho_at_Moon_array, # total mass density at the moon
            n_at_Moon_ions, # ions density at the moon orbit cm^-3
            n_at_Moon_electrons, # electrons density at the moon orbit cm^-3 
            units
            ) = density_and_scale_height_of_CS_calculation(r_N, theta_N, phi_N, B_total_N,
                                   r_S, theta_S, phi_S, B_total_S,
                                   E_array,
                                   z_lim_N_optimized, r_0,
                                   longitude_observed_MAW,
                                   longitude_observed_TEB,
                                   rho_0_volumetric_first_guess = rho_0_volumetric_first_guess,
                                   rotation_rate = rotation_rate_callisto,
                                   jovicentric_equator = False,
                                   centrifugal_equator = True,
                                   magnetic_equator = False,
                                   verbose = True,
                                   disk = False,
                                   torus = True,
                                   delta_L = delta_L)


In [ ]:
print(H_array[0][0], # Height Scale in Rp
            rho_0_array, #total mass density in the center of the PS in kg.cm^-3
            n_0_ions_array, # ions density in the PS cm^-3
            n_0_electrons, # electrons density in the PS cm^-3
            longitude_TEB, # TEB longitude
            longitude_MAW, # MAW longitude
            n_at_Moon_ions, # ions density at the moon orbit cm^-3
            n_at_Moon_electrons, # electrons density at the moon orbit cm^-3 
            units
            )

## Plot with the best fit parameters

In [ ]:
from plotting_functions import plot_MFL_and_rho_CS

# Centrifugal equator :
# Magnetic equator :
#H = 1.21*R_J
#rho_0_optimized = 3.12E-21 # km/m^3



z_lim_N = 0.96
H = 0.94*R_J
rho_0_optimized = 4.10E-21 # km/m^3


if jovicentric_equator:
        (x_N,y_N,z_N) = spherical_to_cartesian(r_N, theta_N, phi_N)
        (x_S,y_S,z_S) = spherical_to_cartesian(r_S, theta_S, phi_S)
else:
    if centrifugal_equator:
        theta_d = 9.3*1/3 # Centrifugal Equator tilt
    if magnetic_equator:
        theta_d = 9.3 
    phi_d = 204.2

    theta_N_transformed = 90-(theta_d*numpy.cos((phi_d-(360-phi_N))*numpy.pi/180.)+(90-theta_N))
    theta_S_transformed = 90-(theta_d*numpy.cos((phi_d-(360-phi_S))*numpy.pi/180.)+(90-theta_S))

(x_N,y_N,z_N) = spherical_to_cartesian(r_N, theta_N_transformed, phi_N)
(x_S,y_S,z_S) = spherical_to_cartesian(r_S, theta_S_transformed, phi_S)

equatorial_radius_N = numpy.sqrt(x_N**2 + y_N**2)
equatorial_radius_S = numpy.sqrt(x_S**2 + y_S**2)
mask_CS_N = (z_N <= z_lim_N*R_J) & (equatorial_radius_N > 10*R_J) # Second condition is to be sure no points close to the planet are taken
mask_CS_S = equatorial_radius_S >= equatorial_radius_N[mask_CS_N].min()


x_vals_CS = numpy.concatenate((x_N[mask_CS_N], x_S[mask_CS_S]))
y_vals_CS = numpy.concatenate((y_N[mask_CS_N], y_S[mask_CS_S]))
z_vals_CS = numpy.concatenate((z_N[mask_CS_N], z_S[mask_CS_S]))
equatorial_vals = numpy.concatenate((equatorial_radius_N[mask_CS_N], equatorial_radius_S[mask_CS_S]))

plot_MFL_and_rho_CS(x_N/R_J, y_N/R_J, z_N/R_J,
                        x_S/R_J, y_S/R_J, z_S/R_J,
                        mask_CS_N, mask_CS_S,
                        r_0, H, n_0_electrons[0][0], disk = False,
                        cb_title = r'$\rho$ (cm$^{-3}$)'
                        )
